<a href="https://colab.research.google.com/github/aiAintEasy-HolyCalamity/PersonalFinanceSample/blob/main/PersonalFinanceSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Real-Life RL Example - Personal Finance Assistant**

*Scenario: Helping a person manage bills, spending, and saving.*

**States:**
* Current balance
* Pending bills
* Remaining days in the month

**Actions:**
* Pay bill early
* Delay payment
* Save money
* Spend on a want

**Rewards:**
* Pay bills on time → +10
* Avoid overdraft → +20
* Late fee → -15
* Overdraft → -50
* Extra savings → +5

**RL Benefit:**
* Learns spending patterns over time
* Recommends best times to pay bills or save
* Prevents overdraft while balancing wants vs needs

In [1]:

import numpy as np
import pandas as pd
import random

#Goal - Having more money​

# Sample budget and priority table from the images​
expenses = [
	{'Category': 'Rent', 'Amount': 800, 'Priority': 'High'},
	{'Category': 'Car Payment', 'Amount': 250, 'Priority': 'High'},
	{'Category': 'Utilities', 'Amount': 100, 'Priority': 'High'},
	{'Category': 'Netflix', 'Amount': 15, 'Priority': 'Medium'},
	{'Category': 'Groceries', 'Amount': 300, 'Priority': 'High'},
	{'Category': 'Dining Out', 'Amount': 100, 'Priority': 'Low'},
	{'Category': 'Medical/Health', 'Amount': 100, 'Priority': 'High'},
	{'Category': 'Unexpected Repair', 'Amount': 300, 'Priority': 'High'},
	{'Category': 'Savings Goal', 'Amount': 200, 'Priority': 'High'},
	{'Category': 'Extra Bill', 'Amount': 150, 'Priority': 'Low'}
]
states = ['High Balance', 'Low Balance', 'Safe']
actions = ['Pay', 'Save', 'Spend']

# Initialize Q-Table​
Q = np.zeros((len(states), len(actions)))
gamma = 0.9
alpha = 0.1

# Define reward logic based on priority​
reward_lookup = {
	'High': 30, # Paying important bills is highly valuable​
	'Medium': 15,
	'Low': 5,
	'Save': 2, 	# Lowered to reflect realistic bank interest​
	'Spend': -25 # Increased penalty for overspending​
}

# Simulation​
for episode in range(500):
	state = random.choice(states)
	balance = 2000 # Starting monthly balance
	for step in range(12):
		action = random.choice(actions)

		# Reward computation based on action​
		if action == 'Save':
			reward = reward_lookup['Save']
			balance += 100

		elif action == 'Spend':
			reward = reward_lookup['Spend']
			balance -= 100

		elif action == 'Pay':

			# Randomly select an expense to pay based on priority​
			if expenses:
				expense = random.choice(expenses)
				balance -= expense['Amount']
				reward = reward_lookup.get(expense['Priority'], -10)

			else:
				reward = -10 # No expenses, apply mild penalty​

		# Adjust reward based on balance state​
		if balance < 0:
			reward -= 30 # Penalty for overdraft​
			next_state = 'Low Balance'

		elif balance > 1500:
			next_state = 'High Balance'

		else:
			next_state = 'Safe'
		s_idx = states.index(state)
		a_idx = actions.index(action)
		ns_idx = states.index(next_state)

		# Q-learning update​
		Q[s_idx][a_idx] = Q[s_idx][a_idx] + alpha * (
			reward + gamma * np.max(Q[ns_idx]) - Q[s_idx][a_idx]
		)
		state = next_state

# Convert Q-table to DataFrame for easy display​
q_table_data = []

for i, state in enumerate(states):
	q_table_data.append({
		'State': state,
		'Pay': round(Q[i][0], 2),
		'Save': round(Q[i][1], 2),
		'Spend': round(Q[i][2], 2)
	})
q_table_df = pd.DataFrame(q_table_data)
print(q_table_df.to_string(index=False))

# Add basic test case check​
print("\nBasic Check: Q-table shape should be (3 states x 3 actions)")
print("Q-table shape:", Q.shape)
assert Q.shape == (3, 3), "Q-table shape is incorrect."
print("Test Passed: Q-table shape is correct.")

       State    Pay   Save  Spend
High Balance 224.29 201.92 175.33
 Low Balance 207.32 194.16 171.19
        Safe 223.82 203.96 176.78

Basic Check: Q-table shape should be (3 states x 3 actions)
Q-table shape: (3, 3)
Test Passed: Q-table shape is correct.
